# Import

In [1]:
import sys
sys.path.append(r"D:\data analysis\code\WBI_analysis")  # 例如 r"C:\Users\YourName\Project"
import AnalysisMethod as am
import pandas as pd
import os
import numpy as np
import WBIFunctions as WBI
from datetime import datetime
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering


# Function Definition

In [2]:
def plot_quies_mot_params(df):
    fig, ax = plt.subplots(5,1, figsize=(12,8), sharex=True)
    WBI.plot_binary_background(ax[0], df, "forward", color='blue')
    WBI.plot_binary_background(ax[0], df, "quies_pc", color='red')
    ax[0].set_title('forward')
    WBI.plot_binary_background(ax[1], df, "turn_pc", color='blue')
    WBI.plot_binary_background(ax[1], df, "quies_pc", color='red')
    ax[1].set_title('turn_pc')
    ax[2].plot(df['Vol_Time'], df['sm_velocity'], lw=2)
    WBI.plot_binary_background(ax[2], df, "quies_pc", color='red')
    ax[2].set_title('sm_velocity')
    ax[3].plot(df['Vol_Time'], df['sm_speed'], lw=2)
    WBI.plot_binary_background(ax[3], df, "quies_pc", color='red')
    ax[3].set_title('sm_speed')
    ax[4].plot(df['Vol_Time'], df['sm_CTX'], lw=2)
    WBI.plot_binary_background(ax[4], df, "quies_pc", color='red')
    ax[4].set_title('sm_CTX')
    plt.show()

In [3]:
def plot_quies_mot_params(df):
    fig, ax = plt.subplots(2,1, figsize=(12,2), sharex=True)
    ax[0].plot(df['Vol_Time'], df['forward'], lw=2)
    WBI.plot_binary_background(ax[0], df, "forward", color='blue')
    # WBI.plot_binary_background(ax[0], df, "quies_pc", color='red')
    ax[0].set_title('forward')
    # WBI.plot_binary_background(ax[1], df, "turn_pc", color='blue')
    # WBI.plot_binary_background(ax[1], df, "quies_pc", color='red')
    # ax[1].set_title('turn_pc')
    ax[1].plot(df['Vol_Time'], df['quies_pc'], lw=2)
    WBI.plot_binary_background(ax[1], df, "quies_pc", color='red')
    ax[1].set_title('quies')
    plt.show()

In [4]:
def Single_WBI_Processing(p_f_all, f_folder):
    p_f = p_f_all+'\\'+f_folder

    # (与荧光对时的)行为数据导入，不需要重新分析处理的
    '''激光段的行为数据,帧率为30Hz左右'''
    mot_mid_pkl = [f for f in os.listdir(p_f) if 'MotionMidlineMatchVol.pkl' in f]
    mot_mid_csv = [f for f in os.listdir(p_f) if 'MotionMidlineMatchVol.csv' in f]
    if len(mot_mid_pkl):
        print('read pickle')
        df_mot_vol = pd.read_pickle(os.path.join(p_f, mot_mid_pkl[0])).reset_index(drop=True)
    elif len(mot_mid_csv):
        df_mot_vol = pd.read_csv(os.path.join(p_f, mot_mid_csv[0]))
    else:
        print('No df_mot_midline')

    # 钙信号导入
    calcium_intensity= np.load(os.path.join(p_f, 'calcium_intensity.npy'))
    # 根据mask列作预处理

    mask = df_mot_vol['mask'].values.astype(bool)   # True 表示要置 NaN
    calcium_intensity[:, mask] = np.nan


    # save_p = p_f.split('calcium_intensity.npy')[0]
    print('文件大小:neuron*timestamp',calcium_intensity.shape)
    # 平滑calcium signal： 均值滤波器 (box filter),处理nan值
    # scale = 1.5
    # for i in range(calcium_intensity.shape[0]):
    #     calcium_intensity[i] = (cv2.blur(calcium_intensity[i], (1, 7))*scale)[:,0]
    scale = 1.5
    for i in range(calcium_intensity.shape[0]):
        row = calcium_intensity[i]
        mean_val = np.nanmean(row)
        row_no_nan = np.where(np.isnan(row), mean_val, row)
        smoothed = cv2.blur(row_no_nan.reshape(-1, 1), (7, 1)) * scale
        calcium_intensity[i] = smoothed[:, 0]
    calcium_intensity[:, mask] = np.nan
    # 求前进速度和速率
    df_mot_vol["head_velocity"] = df_mot_vol.apply(WBI.signed_norm, axis=1)
    df_mot_vol["head_speed"] = df_mot_vol["head_velocity"].abs()
    # 连续运动变量平滑
    window_size = 15
    # 计算移动平均值
    df_mot_vol['sm_velocity'] = df_mot_vol['head_velocity'].rolling(window=window_size, min_periods=1).mean()
    df_mot_vol['sm_speed'] = df_mot_vol['head_speed'].rolling(window=window_size, min_periods=1).mean()
    # # 平滑ctx
    window_size = 15
    # 计算移动平均值
    df_mot_vol['sm_CTX'] = df_mot_vol['CTX_left'].rolling(window=window_size, min_periods=1).mean()
    df_mot_vol['sm_ang'] = df_mot_vol['ang_velocity'].rolling(window=window_size, min_periods=1).mean()

    # 定义图片保存文件夹路径
    now = datetime.now()
    hour_str = now.strftime("%Y-%m-%d-%H")
    folder_path = os.path.join(p_f, hour_str+'_AnalysisFigs')
    # 检查文件夹是否存在
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)  # 创建文件夹
        print(f"文件夹 '{folder_path}' 已创建！")
    else:
        print(f"文件夹 '{folder_path}' 已存在！")
    signal_save_path = folder_path+'\\HierClustering'
    os.makedirs(signal_save_path, exist_ok=True)
    
    choose_index = 0
    thresh = 5
    links=['ward','average','average','complete']
    affs=['euclidean','cosine','cityblock','cosine']
    vmin=-0.5
    vmax=1
    smooth_kernel = 10
    print('calcium_intensity.shape',calcium_intensity.shape)

    valid_timepoints = ~mask
    calcium_valid = calcium_intensity[:, valid_timepoints]
    # 使用 np.corrcoef(calcium_intensity) 计算神经元钙信号的相关性矩阵
    # 聚类：使用 am.cluster 和指定的链接方式 (link=links[choose_index]) 
    # 和距离度量 (aff=affs[choose_index]) 对相关性矩阵进行聚类。
    idx=am.cluster(np.corrcoef(calcium_valid),link=links[choose_index],aff=affs[choose_index])
    # idx为聚类之后的索引
    bound=np.cumsum(am.GetBound(np.corrcoef(calcium_valid),link=links[choose_index],aff=affs[choose_index],
                                threshold=thresh).astype(int))
    # 调用 am.GetBound 计算矩阵分区的边界（bound），通过 np.cumsum 累积求和获取完整的边界数组
    print('边界', bound)

    calcium_valid = calcium_valid[idx]
    calcium_intensity = calcium_intensity[idx]
    # calcium_intensity_smd = calcium_intensity.copy()
    # 平滑数据
    if smooth_kernel:
        for i,k in enumerate(calcium_valid):
            calcium_valid[i] = cv2.blur(k,(1,smooth_kernel))[:,0]
    # 计算相关性矩阵
    w_p2m = np.corrcoef(calcium_valid)

    # 聚类并将聚类的结果画在相关性矩阵旁边
    link = links[choose_index]
    aff = affs[choose_index]
    model = AgglomerativeClustering(distance_threshold=0, n_clusters=None,linkage=link, affinity=aff)
    model = model.fit(w_p2m)
    font_size = 100

    '''输入事件开始index列表'''
    neuron_ids = np.arange(calcium_valid.shape[0])
    df_mot_vol['forward_quies'] = df_mot_vol['forward'].copy()
    df_mot_vol.loc[df_mot_vol['quies_pc'] == 1,'forward_quies'] = 2
    col_draw = ['sm_velocity','sm_speed', 'sm_ang', 'sm_CTX','curvature', 'forward_quies','turn_pc','turn_cor']
    # df_mot_valid = df_mot_vol[df_mot_vol['mask']==0]
    WBI.calcium_heatmap(calcium_intensity,df_mot_vol, col_draw,  neuron_ids,model,w_p2m, show_id_stride=10,
                    show_vol_stride=500, heatmap_range=(0,0.6),wspace=0.06, hspace=0.2,bound_cluster=bound,
                    unit_w=0.03, unit_h = 0.8, cal_height_ratio=30, smooth_kernel=smooth_kernel,
                    font_size=font_size, font_color='black', idx = idx, vmin = vmin, vmax = vmax,
                        threshold=thresh, xlabel='Neuron Index',level=35, signal_save_path=signal_save_path)
    '''注意这里是通过上一步排序后的钙信号数据！'''

    calcium_intensity_sort = calcium_intensity.copy()
    # 写出重排序后的钙信号
    np.save(folder_path+'\\calcium_intensity_sorted.npy', calcium_intensity_sort)
    block_path = signal_save_path+'\\BlockHighlight'
    os.makedirs(block_path, exist_ok=True)
    calcium_dict = {i: calcium_intensity_sort[i] for i in range(len(calcium_intensity_sort))}
    # WBI.block_visualize_traces(block_path,calcium_intensity_sort, calcium_dict,df_mot_vol, bound, smooth=True, sigma=0.3,delta_y=1.2)
    calcium_intensity_sort_T = calcium_intensity_sort.T


    # 创建列名
    col_neuron_names = [f"neuron{i+1}" for i in range(calcium_intensity_sort_T.shape[1])]
    df_calcium = pd.DataFrame(calcium_intensity_sort_T, columns=col_neuron_names)
    # 按照行索引合并
    df_cal_motion_org = pd.concat([df_calcium, df_mot_vol], axis=1)
    # 将masl=1的部分去掉
    # df_cal_motion.loc[df_cal_motion['mask']==1,:]=np.nan
    df_cal_motion = df_cal_motion_org.loc[df_cal_motion_org['mask']==0,:]
    corr_folder = folder_path+'\\CorrAnalysis'
    os.makedirs(corr_folder, exist_ok=True)
    p_corr_ls = []  # 用于收集相关性分析的df的列表

    # 根据静息处理运动参数列
    # cols_ls = ['forward', "turn_pc", "turn_cor"]
    cols_ls = ['forward']
    df_cal_motion.loc[df_cal_motion['quies_pc']==1, cols_ls] = np.nan
    cols_ls_2 = ['sm_velocity','sm_speed','sm_CTX']
    df_cal_motion.loc[df_cal_motion['quies_pc']==1, cols_ls_2] = 0
    cols_ls_3 = ['sm_ang','curvature']
    df_cal_motion.loc[df_cal_motion['quies_pc']==1, cols_ls_3] = np.nan
    # plot_quies_mot_params(df_cal_motion)
    # return None
    # 前后后退转换
    # 开始
    t_win = 25
    neu_cols = [col for col in df_cal_motion.columns if 'neuron' in col]
    rev_start_idx = WBI.get_event_start_idx(df_cal_motion, 'forward', 'start')
    df_rev_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'forward', rev_start_idx, neu_cols,
                                            test_window = t_win, event_note='RevStart')  # 注意帧率为3左右
    if df_rev_s_p is not None:
        df_rev_s_c = WBI.multi_compar(df_rev_s_p,adj_method='fdr_bh')
        df_rev_s_c['sign'] = 0
        df_rev_s_c.loc[df_rev_s_c['p_cor']<=0.05,'sign'] = 1
        p_corr_ls.append(df_rev_s_c)
        sel_n_s = df_rev_s_c[(df_rev_s_c['p_cor']<=0.05)]['Neuron'].unique()
        rev_folder = corr_folder+'\\Reverse'
        if len(sel_n_s):
            WBI.plot_event_aligned_traces_combined(df_cal_motion,'forward',df_rev_s_c, sel_n_s, rev_start_idx,
                                        pre_window = t_win,post_window=t_win, 
                                            sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                            heatmap_range = (0,0.6),save_title_str='ReverseStart',x_label_str='Reverse Start',
                                            cmap='jet', unify_yaxis=True, save_path=rev_folder)
            # 作折线图
            hl_col_ls= ['forward']
            print(sel_n_s)
            WBI.plot_lines_neural_activity_disc(df_cal_motion, sel_n_s,df_rev_s_c,"RevStart",Event_col="EventNote",
                                hl_col =hl_col_ls,hl_color=[ "#1BA4FF"], num_bins=5,
                                    title=f"NeuCorrWithRevStartHighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                color='#2878B5', x_label='', y_label='',sigma = 5,
                                strip_color='#9AC9DB', label_r=False, p_f=rev_folder)
    # 结束
    t_win = 20
    rev_end_idx = WBI.get_event_start_idx(df_cal_motion, 'forward', 'end')
    df_rev_e_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'forward', rev_end_idx,
                                                neu_cols,test_window = t_win, event_note='RevEnd')  # 注意帧率为3左右
    if df_rev_e_p is not None:
        df_rev_e_c = WBI.multi_compar(df_rev_e_p,adj_method='fdr_bh')
        df_rev_e_c['sign'] = 0
        df_rev_e_c.loc[df_rev_e_c['p_cor']<=0.05,'sign'] = 1
        p_corr_ls.append(df_rev_e_c)
        sel_n_e = df_rev_e_c[(df_rev_e_c['p_cor']<=0.05)]['Neuron'].unique()

        rev_folder = corr_folder+'\\Reverse'
        if len(sel_n_e):
            WBI.plot_event_aligned_traces_combined(df_cal_motion,'forward',df_rev_e_c, sel_n_e, rev_end_idx,
                                        pre_window = t_win,post_window=t_win, 
                                            sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                            heatmap_range = (0,0.6),save_title_str='ReverseEnd',x_label_str='Reverse End',
                                            cmap='jet', unify_yaxis=True, save_path=rev_folder)
            hl_col_ls = ['forward']
            WBI.plot_lines_neural_activity_disc(df_cal_motion, sel_n_e,df_rev_e_c,'RevEnd',Event_col="EventNote",
                                hl_col =hl_col_ls,hl_color=[ "#1BA4FF"], num_bins=5,
                                    title=f"NeuCorrWithRevEndHighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                color='#2878B5', x_label='', y_label='',sigma = 5,
                                strip_color='#9AC9DB', label_r=False, p_f=rev_folder)

    # omega turn(coiling开始)
    # 开始
    t_win = 10
    # neu_cols = [col for col in df_cal_motion.columns if 'neuron' in col]
    c_start_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_pc', 'start')
    df_turn_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_pc', c_start_idx,
                                                neu_cols,test_window = t_win, event_note='CoilingStart')  # 注意帧率为3左右
    if df_turn_s_p is not None:
        df_turn_s_c = WBI.multi_compar(df_turn_s_p,adj_method='fdr_bh')
        
        df_turn_s_c['sign'] = 0
        df_turn_s_c.loc[df_turn_s_c['p_cor']<=0.05,'sign'] = 1
        p_corr_ls.append(df_turn_s_c)
        sel_n_turn = df_turn_s_c[(df_turn_s_c['p_cor']<=0.05)]['Neuron'].unique()

        turn_folder = corr_folder+'\\OmegaTurn'
        if len(sel_n_turn):
            WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_pc',df_turn_s_c, sel_n_turn, c_start_idx,
                                        pre_window = t_win,post_window=t_win, 
                                            sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                            heatmap_range = (0,0.6),save_title_str='CoilingStart',x_label_str='Coiling Start',
                                            cmap='jet', unify_yaxis=True, save_path=turn_folder)
            hl_col_ls = ['forward','turn_pc']
            WBI.plot_lines_neural_activity_disc(df_cal_motion, sel_n_turn,df_turn_s_c,"CoilingStart",Event_col="EventNote",
                                    hl_col = hl_col_ls,hl_color=[ "#1BA4FF","#F0EC61",], num_bins=5,
                                    title=f"NeuCorrWithCoilingStartHighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                color='#2878B5', x_label='', y_label='',sigma = 5,
                                strip_color='#9AC9DB', label_r=False, p_f=turn_folder)
    # omega turn (校正后的)
    # 开始
    t_win = 10
    oturn_start_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_cor', 'start')
    df_oturn_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_cor', oturn_start_idx,
                                                neu_cols,test_window = t_win, event_note='OmegaStart')  # 注意帧率为3左右
    if df_oturn_s_p is not None:
        df_oturn_s_c = WBI.multi_compar(df_oturn_s_p,adj_method='fdr_bh')
        sel_n_oturn = df_oturn_s_c[(df_oturn_s_c['p_cor']<=0.05)]['Neuron'].unique()
        df_oturn_s_c['sign'] = 0
        df_oturn_s_c.loc[df_oturn_s_c['p_cor']<=0.05,'sign'] = 1
        p_corr_ls.append(df_oturn_s_c)
        turn_folder = corr_folder+'\\OmegaTurn'
        if len(sel_n_oturn):
            WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_cor',df_oturn_s_c, sel_n_oturn, oturn_start_idx,
                                        pre_window = t_win,post_window=t_win, 
                                            sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                            heatmap_range = (0,0.6),save_title_str='OmegaTurnStart',x_label_str='OmegaTurn Start',
                                            cmap='jet', unify_yaxis=True, save_path=turn_folder)
            hl_col_ls = ['forward','turn_pc','turn_cor']
            WBI.plot_lines_neural_activity_disc(df_cal_motion, sel_n_oturn,df_oturn_s_c,"OmegaStart",Event_col="EventNote",
                                hl_col = hl_col_ls,hl_color=[ "#1BA4FF","#F0EC61","#DF6176"], num_bins=5,
                                    title=f"NeuCorrWithOmegaStartHighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                color='#2878B5', x_label='', y_label='',sigma = 5,
                                strip_color='#9AC9DB', label_r=False, p_f=turn_folder)
    
    # omega turn end
    # 开始
    t_win = 10
    oturn_end_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_cor', 'end')
    df_oturn_e_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_cor', oturn_end_idx,
                                                neu_cols,test_window = t_win, event_note='OmegaEnd')  # 注意帧率为3左右
    if df_oturn_e_p is not None:
        df_oturn_e_c = WBI.multi_compar(df_oturn_e_p,adj_method='fdr_bh')
        df_oturn_e_c['sign'] = 0
        df_oturn_e_c.loc[df_oturn_e_c['p_cor']<=0.05,'sign'] = 1
        p_corr_ls.append(df_oturn_e_c)
        sel_n_oturn_e = df_oturn_e_c[(df_oturn_e_c['p_cor']<=0.05)]['Neuron'].unique()

        turn_folder = corr_folder+'\\OmegaTurn'
        if len(sel_n_oturn_e):
            WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_cor',df_oturn_e_c, sel_n_oturn_e, oturn_end_idx,
                                        pre_window = t_win,post_window=t_win, 
                                            sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                            heatmap_range = (0,0.6),save_title_str='OmegaTurnEnd',x_label_str='OmegaTurn End',
                                            cmap='jet', unify_yaxis=True, save_path=turn_folder)
            hl_col_ls = ['forward','turn_pc','turn_cor']
            WBI.plot_lines_neural_activity_disc(df_cal_motion, sel_n_oturn_e,df_oturn_e_c,"OmegaEnd",Event_col='EventNote',
                                hl_col = hl_col_ls,hl_color=[ "#1BA4FF","#F0EC61","#DF6176"], num_bins=5,
                                    title=f"NeuCorrWithOmegaEndHighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                color='#2878B5', x_label='', y_label='',sigma = 5,
                                strip_color='#9AC9DB', label_r=False, p_f=turn_folder)
    # 离散变量计算相关系数
    beh_col = 'turn_pc'
    p_uplim = 0.05
    r_threshold = 0.25
    df_neu_coil_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                        neuron_col_str='neuron',adj_method = 'fdr_bh' )
    df_neu_coil_p['sign'] = 0
    df_neu_coil_p.loc[(df_neu_coil_p['p_cor']<=p_uplim)&(df_neu_coil_p['true_r'].abs()>r_threshold),'sign'] = 1
    p_corr_ls.append(df_neu_coil_p)
    # 显著的神经元
    sign_neu_ls = df_neu_coil_p[(df_neu_coil_p['sign']==1)&((df_neu_coil_p['Event']==beh_col))]['Neuron'].unique()

    hl_col_ls = ['forward',"turn_pc"]


    if len(sign_neu_ls)>0:
        WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_coil_p,beh_col,
                                hl_col =hl_col_ls,hl_color=[ "#1BA4FF","#DF6176",], num_bins=5,
                                    title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                color='#2878B5', x_label='', y_label='',sigma = 5,
                                strip_color='#9AC9DB', p_f=turn_folder+f'\\{beh_col}_pearson')
    beh_col = 'turn_cor'
    p_uplim = 0.05
    r_threshold = 0.25
    df_neu_omega_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                        neuron_col_str='neuron',adj_method = 'fdr_bh' )
    df_neu_omega_p['sign'] = 0
    df_neu_omega_p.loc[(df_neu_omega_p['p_cor']<=p_uplim)&(df_neu_omega_p['true_r'].abs()>r_threshold),'sign'] = 1
    p_corr_ls.append(df_neu_omega_p)
    # 显著的神经元
    sign_neu_ls = df_neu_omega_p[(df_neu_omega_p['sign']==1)&((df_neu_omega_p['Event']==beh_col))]['Neuron'].unique()
    hl_col_ls = ['forward','turn_pc',"turn_cor"]
    if len(sign_neu_ls)>0:
        WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_omega_p,beh_col=beh_col,
                                    hl_col =hl_col_ls,hl_color=[ "#1BA4FF","#F0EC61","#DF6176",], num_bins=5,
                                        title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                    fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                    color='#2878B5', x_label='', y_label='',sigma = 5,
                                    strip_color='#9AC9DB', p_f=turn_folder+f'\\{beh_col}_pearson')
    
    # 离散变量皮尔逊相关
    beh_col = 'forward'
    p_uplim = 0.05
    r_threshold = 0.25
    df_neu_coil_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                        neuron_col_str='neuron',adj_method = 'fdr_bh' )
    df_neu_coil_p['sign'] = 0
    df_neu_coil_p.loc[(df_neu_coil_p['p_cor']<=p_uplim)&(df_neu_coil_p['true_r'].abs()>r_threshold),'sign'] = 1
    p_corr_ls.append(df_neu_coil_p)
    # 显著的神经元
    sign_neu_ls = df_neu_coil_p[(df_neu_coil_p['sign']==1)&((df_neu_coil_p['Event']==beh_col))]['Neuron'].unique()

    hl_col_ls = ['forward']
    WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_coil_p,beh_col,
                                hl_col =hl_col_ls,hl_color=[ "#1BA4FF"], num_bins=5,
                                    title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                color='#2878B5', x_label='', y_label='',sigma = 5,
                                strip_color='#9AC9DB', label_p = True,label_r = True, p_f=corr_folder+f'\\{beh_col}')
    # 连续变量
    df_neu_con_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=['sm_velocity','sm_speed','sm_ang','curvature','sm_CTX'],
                        neuron_col_str='neuron',adj_method = 'fdr_bh' )
    df_neu_con_p['sign'] = 0
    df_neu_con_p.loc[(df_neu_con_p['p_cor']<=0.05) & (df_neu_con_p['true_r']>= 0.3),'sign'] = 1
    p_corr_ls.append(df_neu_con_p)
    r_threhold = 0.2
    x_ticklabels=[f"{i}" for i in range(1, 1+len(df_neu_con_p['Neuron'].unique()))]
    WBI.plot_correlation_heatmap_nomask_from_df(df_neu_con_p,
                                    row_col = 'Event',
                                    col_col='Neuron',
                                        alpha_original=0.05,
                                        title="Neuron-Behavior Correlation Heatmap",
                                    mask_color='#f0f0f0',   # 灰白
                                    x_ticklabels=x_ticklabels,
                                    r_threshold=r_threhold,
                                    save_path=corr_folder,
                                    save_title='CorrNeuAllContinous',
                                        fs = 12.5)
    df_corr_p = pd.concat(p_corr_ls, axis=0, ignore_index=True)
    df_corr_p['EventNote'] = df_corr_p['EventNote'].fillna(df_corr_p['Event'])
    WBI.plot_correlation_heatmap_nomask_from_df(df_corr_p,
                                    row_col = 'EventNote',
                                    col_col='Neuron',
                                        alpha_original=0.05,
                                        title="Neuron-Behavior Correlation Heatmap",
                                    mask_color='#f0f0f0',   # 灰白
                                    x_ticklabels=x_ticklabels,
                                    r_threshold=r_threhold,
                                    save_path=corr_folder,
                                    save_title='CorrNeuAllVariables',
                                        fs = 12.5)
    beh_sig_ls = ['sm_velocity','sm_speed','sm_ang','curvature','sm_CTX']  # 可视化行为列表
    # beh_sig_ls = ['sm_ang','curvature','sm_CTX']
    for beh_col in beh_sig_ls:
        WBI.visualize_cont_beh_by_neu(df_neu_con_p,df_cal_motion, beh_col, p_uplim=0.05, r_threshold=0.2, p_f_con = corr_folder)    
    # 写出p值文件
    
    df_corr_p['File'] = f_folder
    corr_p_path = corr_folder+f'\\{f_folder}_corr_p_cor.csv'
    df_corr_p.to_csv(corr_p_path, index=False)




In [5]:

def get_inbtw_file(files, start_str = '20250221', end_str='20250830'):
    new_files = []
    for f in files:
        date_str = f.split('_')[0]

        start = datetime.strptime(start_str, "%Y%m%d") 
        end = datetime.strptime(end_str, "%Y%m%d")
        # 将日期字符串转换为datetime对象
        date = datetime.strptime(date_str, "%Y%m%d")
        if (date >= start) & (date <= end):
            new_files.append(f)
    newline = '\n'
    print(f"保留处理文件:\n{newline.join(new_files)}")
    return new_files

# 多文件处理

In [6]:
p_f_all = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done\\'
key_word = ''
nokey_word = '*'
files = [f_p for f_p in os.listdir(p_f_all) if ('redo' not in f_p)&
              ('done' not in f_p)&
             os.path.isdir(os.path.join(p_f_all,f_p))&
             (key_word in f_p)&(nokey_word not in f_p)]
files = get_inbtw_file(files, start_str = '20241201', end_str='20250830')
for file in files:
    print(f'=====开始处理{file}======')
    df_p_cor = Single_WBI_Processing(p_f_all, file)

保留处理文件:
20241219_4.5g-ov_06
20250115_4.5g-24d-ov_08
20250116_4.5g-ov_05
20250221_0g-ov-27.5d_6
20250225_0g-ov-24d_003
20250225_0g-ov-24d_004
20250225_4.5g-ov-24d_002
20250225_4.5g-ov-24d_016
20250312_4.5g-24d-4h_007
20250319_1g-23d-4h_001
20250409_5g-ov-24d_011
20250422_3g-lg9624-3h-24.5d_004
20250422_4.5gNa-lg9624-6h-24d_015
20250425_1g-9624-ov-24h_001
20250609_lg9624-1g-00225-23.5d-ov_008
20250609_lg9624-1g-00225-24d-ov_006
20250805_lg9624-1gNa-24d-ov_009
20250805_lg9624-1gNa-24d-ov_011
20250805_lg9624-1gNa-24d-ov_013
20250823_lg9624-1gNa-24.5d-1h-new_020
20250823_lg9624-1gNa-24.5d-4h-new_016
20250823_lg9624-1gNa-24d-1h-new_013
20250823_lg9624-1gNa-24d-1h-old_006
20250823_lg9624-1gNa-24d-4h-new_022
=====开始处理20241219_4.5g-ov_06======
read pickle
文件大小:neuron*timestamp (89, 1464)
文件夹 'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done\\\20241219_4.5g-ov_06\2025-10-23-15_AnalysisFigs' 已存在！
calcium_intensity.shape (89, 1464)
边界 [53 59 89]
(事次*timestamp)(5, 51)
1->0 转变的索引: [260, 712, 1021, 1125, 1

In [7]:
p_f_all = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\to_do_q\\'
key_word = ''
nokey_word = '*'
files = [f_p for f_p in os.listdir(p_f_all) if ('redo' not in f_p)&
              ('done' not in f_p)&
             os.path.isdir(os.path.join(p_f_all,f_p))&
             (key_word in f_p)&(nokey_word not in f_p)]
files = get_inbtw_file(files, start_str = '20241201', end_str='20250830')
for file in files:
    print(f'=====开始处理{file}======')
    df_p_cor = Single_WBI_Processing(p_f_all, file)

保留处理文件:
20241219_0g-ov_05
20250312_0g-24d-4.5h_003
20250409_1g-ov-24d_005
20250409_5g-ov-24d_014
20250709_lg9624-5gNa-ov-24d_007
20250716_lg9624-1gNa-002-24d-ov_002
20250716_lg9624-1gNa-002-24d-ov_006
20250716_lg9624-5gNa-002-24d-ov_004
20250717_lg9624-5gNa-002-24d-ov_005
20250717_lg9624-5gNa-002-24d-ov_006
20250723_lg9624-5gNa-002-24d-ov_008
20250723_lg9624-5gNa-002-24d-ov_014
20250823_lg9624-1gNa-24.5d-1h-old_008
=====开始处理20241219_0g-ov_05======
read pickle
文件大小:neuron*timestamp (115, 4001)
文件夹 'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\to_do_q\\\20241219_0g-ov_05\2025-10-23-22_AnalysisFigs' 已创建！
calcium_intensity.shape (115, 4001)
边界 [ 37  70  93 115]
(事次*timestamp)(7, 51)
(事次*timestamp)(8, 50)
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
['neuron3' 'neuron4' 'neuron11' 'neuron12' 'neuron14' 'neuron16'
 'neuron24' 'neuron29' 'neuron36' 'neuron40' 'neuron41' 'neuron66'
 'neuron69' 'neuron76' 'neuron92' 'neuron93' 'neuron101']
1200.025860895672
总的时间范围：[71.8071391043

## 单文件处理

In [ ]:
p_f = "Y:\\SZX\\2025_wbi_analysis\\good_WBI\\to_do\\20250225_4.5g-ov-24d_016\\20250225_4.5g-ov-24d_016_MotionMidlineMatchVol.pkl"
df = pd.read_pickle(p_f)

In [ ]:
p_f_all = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done'
p_f = p_f_all+'\\'+'20250221_0g-ov-27.5d_6'

# (与荧光对时的)行为数据导入，不需要重新分析处理的
'''激光段的行为数据,帧率为30Hz左右'''
mot_mid_vol = [f for f in os.listdir(p_f) if 'MotionMidlineMatchVol' in f][0]
if 'pkl' in mot_mid_vol:
    df_mot_vol = pd.read_pickle(os.path.join(p_f, mot_mid_vol)).reset_index(drop=True)
else:
    df_mot_vol = pd.read_csv(os.path.join(p_f, mot_mid_vol))
# 钙信号导入
calcium_intensity= np.load(os.path.join(p_f, 'calcium_intensity.npy'))
# 根据mask列作预处理

mask = df_mot_vol['mask'].values.astype(bool)   # True 表示要置 NaN
calcium_intensity[:, mask] = np.nan


# save_p = p_f.split('calcium_intensity.npy')[0]
print('文件大小:neuron*timestamp',calcium_intensity.shape)
# 平滑calcium signal： 均值滤波器 (box filter),处理nan值
# scale = 1.5
# for i in range(calcium_intensity.shape[0]):
#     calcium_intensity[i] = (cv2.blur(calcium_intensity[i], (1, 7))*scale)[:,0]
scale = 1.5
for i in range(calcium_intensity.shape[0]):
    row = calcium_intensity[i]
    mean_val = np.nanmean(row)
    row_no_nan = np.where(np.isnan(row), mean_val, row)
    smoothed = cv2.blur(row_no_nan.reshape(-1, 1), (7, 1)) * scale
    calcium_intensity[i] = smoothed[:, 0]
calcium_intensity[:, mask] = np.nan
# 求前进速度和速率
df_mot_vol["head_velocity"] = df_mot_vol.apply(WBI.signed_norm, axis=1)
df_mot_vol["head_speed"] = df_mot_vol["head_velocity"].abs()
# 连续运动变量平滑
window_size = 15
# 计算移动平均值
df_mot_vol['sm_velocity'] = df_mot_vol['head_velocity'].rolling(window=window_size, min_periods=1).mean()
df_mot_vol['sm_speed'] = df_mot_vol['head_speed'].rolling(window=window_size, min_periods=1).mean()
# # 平滑ctx
window_size = 15
# 计算移动平均值
df_mot_vol['sm_CTX'] = df_mot_vol['CTX_left'].rolling(window=window_size, min_periods=1).mean()
df_mot_vol['sm_ang'] = df_mot_vol['ang_velocity'].rolling(window=window_size, min_periods=1).mean()

# 定义图片保存文件夹路径
folder_path = os.path.join(p_f, 'AnalysisFigs')
# 检查文件夹是否存在
if not os.path.exists(folder_path):
    os.makedirs(folder_path)  # 创建文件夹
    print(f"文件夹 '{folder_path}' 已创建！")
else:
    print(f"文件夹 '{folder_path}' 已存在！")
signal_save_path = folder_path+'\\HierClustering'
os.makedirs(signal_save_path, exist_ok=True)
choose_index = 0
thresh = 5
links=['ward','average','average','complete']
affs=['euclidean','cosine','cityblock','cosine']
vmin=-0.5
vmax=1
smooth_kernel = 10
print('calcium_intensity.shape',calcium_intensity.shape)

valid_timepoints = ~mask
calcium_valid = calcium_intensity[:, valid_timepoints]
# 使用 np.corrcoef(calcium_intensity) 计算神经元钙信号的相关性矩阵
# 聚类：使用 am.cluster 和指定的链接方式 (link=links[choose_index]) 
# 和距离度量 (aff=affs[choose_index]) 对相关性矩阵进行聚类。
idx=am.cluster(np.corrcoef(calcium_valid),link=links[choose_index],aff=affs[choose_index])
# idx为聚类之后的索引
bound=np.cumsum(am.GetBound(np.corrcoef(calcium_valid),link=links[choose_index],aff=affs[choose_index],
                            threshold=thresh).astype(int))
# 调用 am.GetBound 计算矩阵分区的边界（bound），通过 np.cumsum 累积求和获取完整的边界数组
print('边界', bound)

calcium_valid = calcium_valid[idx]
calcium_intensity = calcium_intensity[idx]
calcium_intensity_smd = calcium_intensity.copy()
# 平滑数据
if smooth_kernel:
    for i,k in enumerate(calcium_valid):
        calcium_valid[i] = cv2.blur(k,(1,smooth_kernel))[:,0]
# 计算相关性矩阵
w_p2m = np.corrcoef(calcium_valid)

# 聚类并将聚类的结果画在相关性矩阵旁边
link = links[choose_index]
aff = affs[choose_index]
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None,linkage=link, affinity=aff)
model = model.fit(w_p2m)
font_size = 100

'''输入事件开始index列表'''
neuron_ids = np.arange(calcium_valid.shape[0])
df_mot_vol['forward_quies'] = df_mot_vol['forward'].copy()
df_mot_vol.loc[df_mot_vol['quies_pc'] == 1,'forward_quies'] = 2
col_draw = ['sm_velocity','sm_speed', 'sm_ang', 'sm_CTX','curvature', 'forward_quies','turn_pc','turn_cor']
df_mot_valid = df_mot_vol[df_mot_vol['mask']==0]
WBI.calcium_heatmap(calcium_intensity,df_mot_vol, col_draw,  neuron_ids,model,w_p2m, show_id_stride=10,
                show_vol_stride=500, heatmap_range=(0,0.6),wspace=0.06, hspace=0.2,bound_cluster=bound,
                unit_w=0.03, unit_h = 0.8, cal_height_ratio=30, smooth_kernel=smooth_kernel,
                font_size=font_size, font_color='black', idx = idx, vmin = vmin, vmax = vmax,
                    threshold=thresh, xlabel='Neuron Index',level=35, signal_save_path=signal_save_path)
'''注意这里是通过上一步排序后的钙信号数据！'''

calcium_intensity_sort = calcium_intensity.copy()
block_path = signal_save_path+'\\BlockHighlight'
os.makedirs(block_path, exist_ok=True)
calcium_dict = {i: calcium_intensity_sort[i] for i in range(len(calcium_intensity_sort))}
WBI.block_visualize_traces(block_path,calcium_intensity_sort, calcium_dict,df_mot_vol, bound, smooth=True, sigma=0.3,delta_y=1.2)
calcium_intensity_sort_T = calcium_intensity_sort.T


# 创建列名
col_neuron_names = [f"neuron{i+1}" for i in range(calcium_intensity_sort_T.shape[1])]
df_calcium = pd.DataFrame(calcium_intensity_sort_T, columns=col_neuron_names)
# 按照行索引合并
df_cal_motion_org = pd.concat([df_calcium, df_mot_vol], axis=1)
# 将masl=1的部分去掉
# df_cal_motion.loc[df_cal_motion['mask']==1,:]=np.nan
df_cal_motion = df_cal_motion_org.loc[df_cal_motion_org['mask']==0,:]
corr_folder = folder_path+'\\CorrAnalysis'
os.makedirs(corr_folder, exist_ok=True)
p_corr_ls = []  # 用于收集相关性分析的df的列表

# 根据静息处理运动参数列
cols_ls = ['forward', "turn_pc", "turn_cor"]
df_cal_motion.loc[df_cal_motion['quies_pc']==1,cols_ls]==np.nan
cols_ls_2 = ['sm_velocity','sm_speed','sm_CTX']
df_cal_motion.loc[df_cal_motion['quies_pc']==1,cols_ls_2]==0
# 前后后退转换
# 开始
t_win = 15
neu_cols = [col for col in df_cal_motion.columns if 'neuron' in col]
rev_start_idx = WBI.get_event_start_idx(df_cal_motion, 'forward', 'start')
df_rev_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'forward', rev_start_idx, neu_cols,
                                        test_window = t_win, event_note='RevStart')  # 注意帧率为3左右
if df_rev_s_p is not None:
    df_rev_s_c = WBI.multi_compar(df_rev_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_rev_s_c)
    sel_n_s = df_rev_s_c[(df_rev_s_c['p_cor']<=0.05)]['Neuron']
    rev_folder = corr_folder+'\\Reverse'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'forward',df_rev_s_c, sel_n_s, rev_start_idx,
                                pre_window = t_win,post_window=t_win, 
                                    sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                    heatmap_range = (0,0.6),save_title_str='ReverseStart',x_label_str='Reverse Start',
                                    cmap='jet', unify_yaxis=True, save_path=rev_folder)



# 结束
t_win = 20
rev_end_idx = WBI.get_event_start_idx(df_cal_motion, 'forward', 'end')
df_rev_e_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'forward', rev_end_idx,
                                            neu_cols,test_window = t_win, event_note='RevEnd')  # 注意帧率为3左右
if df_rev_e_p is not None:
    df_rev_e_c = WBI.multi_compar(df_rev_e_p,adj_method='fdr_bh')
    p_corr_ls.append(df_rev_e_c)
    sel_n_e = df_rev_e_c[(df_rev_e_c['p_cor']<=0.1)]['Neuron']

    rev_folder = corr_folder+'\\Reverse'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'forward',df_rev_e_c, sel_n_e, rev_end_idx,
                                pre_window = t_win,post_window=t_win, 
                                    sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                    heatmap_range = (0,0.6),save_title_str='ReverseEnd',x_label_str='Reverse End',
                                    cmap='jet', unify_yaxis=True, save_path=rev_folder)


# omega turn(coiling开始)
# 开始
t_win = 10
# neu_cols = [col for col in df_cal_motion.columns if 'neuron' in col]
c_start_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_pc', 'start')
df_turn_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_pc', c_start_idx,
                                            neu_cols,test_window = t_win, event_note='CoilingStart')  # 注意帧率为3左右
if df_turn_s_p is not None:
    df_turn_s_c = WBI.multi_compar(df_turn_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_turn_s_c)
    sel_n_turn = df_turn_s_c[(df_turn_s_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_pc',df_turn_s_c, sel_n_turn, c_start_idx,
                                pre_window = t_win,post_window=t_win, 
                                    sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                    heatmap_range = (0,0.6),save_title_str='CoilingStart',x_label_str='Coiling Start',
                                    cmap='jet', unify_yaxis=True, save_path=turn_folder)


# omega turn (校正后的)
# 开始
t_win = 10
oturn_start_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_cor', 'start')
df_oturn_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_cor', oturn_start_idx,
                                            neu_cols,test_window = t_win, event_note='OmegaStart')  # 注意帧率为3左右
if df_oturn_s_p is not None:
    df_oturn_s_c = WBI.multi_compar(df_oturn_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_oturn_s_c)
    sel_n_oturn = df_oturn_s_c[(df_oturn_s_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_cor',df_oturn_s_c, sel_n_oturn, oturn_start_idx,
                                pre_window = t_win,post_window=t_win, 
                                    sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                    heatmap_range = (0,0.6),save_title_str='OmegaTurnStart',x_label_str='OmegaTurn Start',
                                    cmap='jet', unify_yaxis=True, save_path=turn_folder)

# omega turn end
# 开始
t_win = 10
oturn_end_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_cor', 'end')
df_oturn_e_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_cor', oturn_end_idx,
                                            neu_cols,test_window = t_win, event_note='OmegaEnd')  # 注意帧率为3左右
if df_oturn_e_p is not None:
    df_oturn_e_c = WBI.multi_compar(df_oturn_e_p,adj_method='fdr_bh')
    p_corr_ls.append(df_oturn_e_c)
    sel_n_oturn_e = df_oturn_e_c[(df_oturn_e_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_cor',df_oturn_e_c, sel_n_oturn_e, oturn_end_idx,
                                pre_window = t_win,post_window=t_win, 
                                    sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                    heatmap_range = (0,0.6),save_title_str='OmegaTurnEnd',x_label_str='OmegaTurn End',
                                    cmap='jet', unify_yaxis=True, save_path=turn_folder)
# 离散变量
beh_col = 'turn_pc'
p_uplim = 0.05
r_threshold = 0.25
df_neu_coil_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                    neuron_col_str='neuron',adj_method = 'fdr_bh' )
df_neu_coil_p['sign'] = 0
df_neu_coil_p.loc[(df_neu_coil_p['p_cor']<=p_uplim)&(df_neu_coil_p['true_r'].abs()>r_threshold),'sign'] = 1
p_corr_ls.append(df_neu_coil_p)
# 显著的神经元
sign_neu_ls = df_neu_coil_p[(df_neu_coil_p['sign']==1)&((df_neu_coil_p['Event']==beh_col))]['Neuron'].unique()

hl_col_ls = ['forward',"turn_pc"]


if len(sign_neu_ls)>0:
    WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_coil_p,beh_col,
                            hl_col =hl_col_ls,hl_color=[ "#1BA4FF","#DF6176",], num_bins=5,
                                title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                            fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                            color='#2878B5', x_label='', y_label='',sigma = 5,
                            strip_color='#9AC9DB', p_f=corr_folder+f'\\{beh_col}')
beh_col = 'turn_cor'
p_uplim = 0.05
r_threshold = 0.25
df_neu_omega_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                    neuron_col_str='neuron',adj_method = 'fdr_bh' )
df_neu_omega_p['sign'] = 0
df_neu_omega_p.loc[(df_neu_omega_p['p_cor']<=p_uplim)&(df_neu_omega_p['true_r'].abs()>r_threshold),'sign'] = 1
p_corr_ls.append(df_neu_omega_p)
# 显著的神经元
sign_neu_ls = df_neu_omega_p[(df_neu_omega_p['sign']==1)&((df_neu_omega_p['Event']==beh_col))]['Neuron'].unique()

hl_col_ls = ['forward','turn_pc',"turn_cor"]
if len(sign_neu_ls)>0:
    WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_omega_p,beh_col=beh_col,
                                hl_col =hl_col_ls,hl_color=[ "#1BA4FF","#F0EC61","#DF6176",], num_bins=5,
                                    title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                                fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                                color='#2878B5', x_label='', y_label='',sigma = 5,
                                strip_color='#9AC9DB', p_f=corr_folder+f'\\{beh_col}')
# 连续变量
df_neu_con_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=['sm_velocity','sm_speed','sm_ang','curvature','sm_CTX'],
                    neuron_col_str='neuron',adj_method = 'fdr_bh' )
p_corr_ls.append(df_neu_con_p)
r_threhold = 0.2
x_ticklabels=[f"{i}" for i in range(1, 1+len(df_neu_con_p['Neuron'].unique()))]
WBI.plot_correlation_heatmap_nomask_from_df(df_neu_con_p,
                                row_col = 'Event',
                                col_col='Neuron',
                                    alpha_original=0.05,
                                    title="Neuron-Behavior Correlation Heatmap",
                                mask_color='#f0f0f0',   # 灰白
                                x_ticklabels=x_ticklabels,
                                r_threshold=r_threhold,
                                save_path=corr_folder,
                                save_title='CorrNeuAllContinous',
                                    fs = 12.5)
beh_sig_ls = ['sm_velocity','sm_speed','sm_ang','curvature','sm_CTX']  # 可视化行为列表
# beh_sig_ls = ['sm_ang','curvature','sm_CTX']
for beh_col in beh_sig_ls:
    WBI.visualize_cont_beh_by_neu(df_neu_con_p,df_cal_motion, beh_col, p_uplim=0.05, r_threshold=0.2, p_f_con = corr_folder)    
# 写出p值文件
df_corr_p = pd.concat(p_corr_ls, axis=0, ignore_index=True)
df_corr_p['File'] = f_folder
corr_p_path = corr_folder+f'\\{f_folder}_corr_p_cor.csv'
df_corr_p.to_csv(corr_p_path, index=False)

